### Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import math
from math import sqrt
plt.style.use('seaborn-poster')

C:\Users\maria\AppData\Local\Temp\ipykernel_7912\700012485.py:6: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-poster')


### Custos máximos

In [3]:
def custo_eletrico_banho(Sr, potencia_eletrica, custo_eletrico_kwh, dt):
    """Calcula o custo da parte elétrica do banho.

    O custo da parte elétrica do banho é dado pela potência do chuveiro em KW multiplicado pela fração de utilização
    da resistência elétrica Sr, o custo do kWh em reais, e o tempo do banho em horas. Como o tempo é em minutos, 
    divide-se por 60. Como a estratégia de controle utilizada foi split-range para Sr, o valor considerado no cálculo
    para Sr será a área da curva (integral).

    Argumentos:
        Sr (float): Seletor da resistência elétrica do tanque de aquecimento.
        potencial_eletrica (float): Potência elétrica do tanque de aquecimento (chuveiro) em kW.
        custo_eletrico_kwh (float): Custo do kWh da energia em reais por hora.
        dt (float): Passo de tempo.
        
    Retorna:
        custo_eletrico_total (float): Custo da energia elétrica do banho em reais.
    """
    Sr_utilizado = np.trapz(y=Sr, dx=dt)
    custo_eletrico_total = potencia_eletrica * Sr_utilizado * custo_eletrico_kwh / 60

    return custo_eletrico_total

In [7]:
potencia_eletrica = 5.5
custo_eletrico_kwh = 1, 1.25, 1.5, 1.75, 2, 2.25
dt = 0.01
Sr = np.repeat(1, 1400)

for val in custo_eletrico_kwh:
    custo_eletrico_max = custo_eletrico_banho(Sr, potencia_eletrica, val, dt)
    print(f"Custo elétrico do kWh: {val}")
    print(f"Custo elétrico máximo do banho: {custo_eletrico_max}")
    print("")

Custo elétrico do kWh: 1
Custo elétrico máximo do banho: 1.2824166666666665

Custo elétrico do kWh: 1.25
Custo elétrico máximo do banho: 1.6030208333333331

Custo elétrico do kWh: 1.5
Custo elétrico máximo do banho: 1.923625

Custo elétrico do kWh: 1.75
Custo elétrico máximo do banho: 2.2442291666666665

Custo elétrico do kWh: 2
Custo elétrico máximo do banho: 2.564833333333333

Custo elétrico do kWh: 2.25
Custo elétrico máximo do banho: 2.8854374999999997



In [8]:
def custo_gas_banho(Sa, potencia_aquecedor, custo_gas_kg, dt):
    """Calcula o custo do gás do banho.

    A potência de um aquecedor a gás é dada em kcal/h. Considerando um rendimento de 86%, a potência últi será
    a potência multiplicada pelo rendimento. Para saber quantas kcal são fornecidas durante o banho, multiplica-se
    a potência útil pela quantidade de energia gasta de Sa (área da curva) e divide por 60 para tempo em horas.
    O pode calorífico do gás GLP é de 11750 kcal/kg. Se multiplicarmos esse valor pela quantidade de kcal gasta no banho,
    é possível obter a quantidade de gás em kg gasta no banho. Finalmente, o custo em reais é dado pela quantidade de gás em kg
    multiplicada pelo custo do gás em kg/reais.
    
    Referências: 
        https://conteudos.rinnai.com.br/vazao-aquecedor-de-agua-a-gas/#:~:text=A%20pot%C3%AAncia%20do%20aquecedor%20%C3%A9,hora%20(kcal%2Fh)
        https://www.supergasbras.com.br/super-blog/negocio/2021/qual-a-vantagem-do-poder-calorifico-do-glp#:~:text=GLP%3A%2011.750%20Kcal%2Fkg,G%C3%A1s%20Natural%3A%209.400%20Kcal%2Fm%C2%B3
    
    Argumentos:
        Sa (float): selector do aquecimento do boiler.
        potencial_aquecedor (float): Potência do aquecedor (boiler) em kcal/h.
        custo_gas_kg (float): Custo do kg do gás em reais por kg.
        tempo (float): Tempo da ação em minutos.
        df (float): Passo de tempo da simulação.

    Retorna:
        custo_gas_total (float): Custo do gás do banho em reais.
    """
    # Tempo em minutos que o aquecedor fica ligado (Sa = 1):
    # numero_vezes_aquecedor_ligado = np.count_nonzero(Sa == 1) # integrar também?
    # tempo_aquecedor_ligado = tempo * numero_vezes_aquecedor_ligado / (tempo / dt)

    # Quantidade de Sa utilizado:
    Sa_utilizado = np.trapz(y=Sa, dx=dt)

    # Potência útil do aquecedor em kcal/h:
    rendimento = 1
    potencia_util = potencia_aquecedor * rendimento

    # Quantidade de kcal fornecida durante o banho:
    # kcal_fornecida_no_banho = potencia_util * tempo_aquecedor_ligado / 60
    kcal_fornecida_no_banho = potencia_util * Sa_utilizado / 60

    # Poder calorífico do gás em kcal/kg (GLP):
    kg_equivalente_kcal = 11750 

    # Quantidade de gás gasta durante o banho:
    quantidade_gas_kg = kcal_fornecida_no_banho / kg_equivalente_kcal 

    # Custo do gás:
    custo_gas_total = custo_gas_kg * quantidade_gas_kg

    return custo_gas_total 

In [11]:
potencia_aquecedor = 29000
custo_gas_kg = 3
dt = 0.01
Sa = np.repeat(1, 1400)

custo_gas_max = custo_gas_banho(Sa, potencia_aquecedor, custo_gas_kg, dt)
custo_gas_max

1.7264255319148933

In [12]:
def modelo_valvula_saida(xs):
    """Modelo para a vazão válvula de saída (Fs).

    Argumentos:
        xs (float): Abertura da válvula de saída.

    Retorna:
        Fs (float): Vazão da válvula de saída.
    """
    n = 5 * (xs ** 3) * (sqrt(30)) * sqrt(-15 * (xs ** 6) + sqrt(6625 * (xs ** 12) + 640 * (xs ** 6) + 16))
    d = 20 * (xs ** 6) + 1

    Fs = n / d
    return Fs

In [13]:
def custo_agua_banho(Fs, custo_agua_m3, tempo):
    """Calcula o custo de água do banho.

    A quantidade de litros gasta em um banho é dada pela vazão em L/min multiplicada pelo tempo em minutos do banho.
    Depois, divide-se a quantidade de litros por 1000, para obter a quantidade em m3 gasta durante o banho.
    Multiplicando esse valor pelo custo da água em m3/reais, obtém-se o custo em reais da água gasta durante o banho.

    Argumentos:
        xs (float): Vazão da válvula de saída.
        custo_agua_m3 (float): Custo da água em m3 por reais. 
        tempo (float): Tempo da ação em minutos.

    Retorna:
        custo_agua_total: Custo da água do banho em reais.
    """
    # integrar a vazão da água
    custo_agua_total = ((Fs * tempo ) / 1000) * custo_agua_m3 

    return custo_agua_total

In [14]:
xs = 0.99
Fs = modelo_valvula_saida(0.99)
Fs

10.92317131464205

In [15]:
custo_agua_m3 = 4
tempo = 14

custo_agua_max = custo_agua_banho(Fs, custo_agua_m3, tempo)
custo_agua_max

0.6116975936199548

### Recompensa

In [16]:
def calculo_iqb(Ts, Fs):
    """Calcula o índice de qualidade do banho (iqb).
    
    Argumentos:
        Ts (float): Temperatura de saída do chuveiro.
        Fs (float): Vazão da válvula de saída.

    Retorna:
        iqb (float): Índice de qualidade do banho.
    """
    iqb = (1 / math.e) * math.exp((1 - ((Ts - 38 + 0.02 * Fs) / 2) ** 2) * np.power((0.506 + math.log10(math.log10((10000 * np.sqrt(Fs)) / (10 + Fs + 0.004 * np.power(Fs, 4))))), 20))

    if np.isnan(iqb) or iqb == None or np.isinf(abs(iqb)) or iqb < 0:
        iqb = 0
    if iqb > 1:
        iqb = 1
        
    return iqb

In [26]:
def calculo_recompensa(Ts, Fs, Sr, Sa):
    
    iqb = calculo_iqb(Ts, Fs)
    print(f"IQB: {iqb}")
    
    dt = 0.01
    
    potencia_eletrica = 5.5
    custo_eletrico_kwh = 1
    custo_eletrico = custo_eletrico_banho(Sr, potencia_eletrica, custo_eletrico_kwh, dt)
    print(f"Custo elétrico: {custo_eletrico}")
    
    potencia_aquecedor = 29000
    custo_gas_kg = 3
    custo_gas = custo_gas_banho(Sa, potencia_aquecedor, custo_gas_kg, dt)
    print(f"Custo gás: {custo_gas}")
    
    custo_agua_m3 = 4
    tempo = 2
    custo_agua = custo_agua_banho(Fs, custo_agua_m3, tempo)
    print(f"Custo água: {custo_agua}")
    
    reward = 5 * iqb - 2 * custo_eletrico - custo_gas - custo_agua
    print(f"Recompensa: {reward}")
    print("")

In [27]:
for i in range(0, 11):
    Ts = 37.5
    Fs = 4.5
    Sr1 = np.repeat(1, 100 + 10 * i)
    Sr2 = np.repeat(0, 100 - 10 * i)
    Sr = np.concatenate((Sr1, Sr2), axis=None)
    Sa1 = np.repeat(1, 100 + 10 * i)
    Sa2 = np.repeat(0, 100 - 10 * i)
    Sa = np.concatenate((Sa1, Sa2), axis=None)
    
    calculo_recompensa(Ts, Fs, Sr, Sa)

IQB: 0.9583508400688454
Custo elétrico: 0.09120833333333332
Custo gás: 0.12278723404255319
Custo água: 0.036
Recompensa: 4.450550299635008

IQB: 0.9583508400688454
Custo elétrico: 0.10037499999999998
Custo gás: 0.13512765957446804
Custo água: 0.036
Recompensa: 4.41987654076976

IQB: 0.9583508400688454
Custo elétrico: 0.10954166666666665
Custo gás: 0.14746808510638296
Custo água: 0.036
Recompensa: 4.389202781904511

IQB: 0.9583508400688454
Custo elétrico: 0.11870833333333333
Custo gás: 0.15980851063829787
Custo água: 0.036
Recompensa: 4.358529023039264

IQB: 0.9583508400688454
Custo elétrico: 0.12787500000000002
Custo gás: 0.17214893617021276
Custo água: 0.036
Recompensa: 4.327855264174016

IQB: 0.9583508400688454
Custo elétrico: 0.13704166666666667
Custo gás: 0.18448936170212765
Custo água: 0.036
Recompensa: 4.2971815053087665

IQB: 0.9583508400688454
Custo elétrico: 0.14620833333333333
Custo gás: 0.19682978723404254
Custo água: 0.036
Recompensa: 4.266507746443518

IQB: 0.9583508400688

In [28]:
for i in range(0, 11):
    Ts = 35
    Fs = 4.5
    Sr1 = np.repeat(1, 100 + 10 * i)
    Sr2 = np.repeat(0, 100 - 10 * i)
    Sr = np.concatenate((Sr1, Sr2), axis=None)
    Sa1 = np.repeat(1, 100 + 10 * i)
    Sa2 = np.repeat(0, 100 - 10 * i)
    Sa = np.concatenate((Sa1, Sa2), axis=None)
    
    calculo_recompensa(Ts, Fs, Sr, Sa)

IQB: 0.12046175941538041
Custo elétrico: 0.09120833333333332
Custo gás: 0.12278723404255319
Custo água: 0.036
Recompensa: 0.26110489636768225

IQB: 0.12046175941538041
Custo elétrico: 0.10037499999999998
Custo gás: 0.13512765957446804
Custo água: 0.036
Recompensa: 0.23043113750243407

IQB: 0.12046175941538041
Custo elétrico: 0.10954166666666665
Custo gás: 0.14746808510638296
Custo água: 0.036
Recompensa: 0.19975737863718585

IQB: 0.12046175941538041
Custo elétrico: 0.11870833333333333
Custo gás: 0.15980851063829787
Custo água: 0.036
Recompensa: 0.16908361977193756

IQB: 0.12046175941538041
Custo elétrico: 0.12787500000000002
Custo gás: 0.17214893617021276
Custo água: 0.036
Recompensa: 0.1384098609066893

IQB: 0.12046175941538041
Custo elétrico: 0.13704166666666667
Custo gás: 0.18448936170212765
Custo água: 0.036
Recompensa: 0.1077361020414411

IQB: 0.12046175941538041
Custo elétrico: 0.14620833333333333
Custo gás: 0.19682978723404254
Custo água: 0.036
Recompensa: 0.0770623431761929

IQ

In [29]:
for i in range(0, 11):
    Ts = 38
    Fs = 4.5
    Sr1 = np.repeat(1, 100 + 10 * i)
    Sr2 = np.repeat(0, 100 - 10 * i)
    Sr = np.concatenate((Sr1, Sr2), axis=None)
    Sa1 = np.repeat(1, 100 + 10 * i)
    Sa2 = np.repeat(0, 100 - 10 * i)
    Sa = np.concatenate((Sa1, Sa2), axis=None)
    
    calculo_recompensa(Ts, Fs, Sr, Sa)

IQB: 0.9974403748301958
Custo elétrico: 0.09120833333333332
Custo gás: 0.12278723404255319
Custo água: 0.036
Recompensa: 4.64599797344176

IQB: 0.9974403748301958
Custo elétrico: 0.10037499999999998
Custo gás: 0.13512765957446804
Custo água: 0.036
Recompensa: 4.615324214576511

IQB: 0.9974403748301958
Custo elétrico: 0.10954166666666665
Custo gás: 0.14746808510638296
Custo água: 0.036
Recompensa: 4.584650455711263

IQB: 0.9974403748301958
Custo elétrico: 0.11870833333333333
Custo gás: 0.15980851063829787
Custo água: 0.036
Recompensa: 4.553976696846015

IQB: 0.9974403748301958
Custo elétrico: 0.12787500000000002
Custo gás: 0.17214893617021276
Custo água: 0.036
Recompensa: 4.523302937980767

IQB: 0.9974403748301958
Custo elétrico: 0.13704166666666667
Custo gás: 0.18448936170212765
Custo água: 0.036
Recompensa: 4.492629179115518

IQB: 0.9974403748301958
Custo elétrico: 0.14620833333333333
Custo gás: 0.19682978723404254
Custo água: 0.036
Recompensa: 4.46195542025027

IQB: 0.997440374830195